### Import classes from autoqchem

In [1]:
from autoqchem.molecule import molecule
from autoqchem.sge_manager import sge_manager
from autoqchem.draw_utils import draw

In [2]:
! conda list auto-qchem

# packages in environment at /opt/anaconda3/envs/autoqchem-test:
#
# Name                    Version                   Build  Channel
auto-qchem                1.3.2                     dev_0    <develop>


### Set the level of logging
Autoqchem prints out log messages, while the logger has various verbosity levels that you can set. If you are running for the first time "INFO" is good, once you are comfortable you can switch it to "WARNING" or "ERROR".

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)

We will use a single smiles string for this test

In [ ]:
smiles_str_list = ['CC', 'CCC']

### Initialize the molecule from smiles
default <code>num_conf=5</code>, which generates 5 conformers for each molecule.
for simple molecules this is enough, but you can change it with <code>num_conf</code>

In [ ]:
mols = [molecule(s) for s in smiles_str_list]
#mols = [molecule(s, num_conf=10) for s in smiles_str_list]

Side note: sometimes rdkit cannot handle the conformer generation of large molecules, especially if they have long alkyl chains. e.g.

In [6]:
big_s = 'CCCCCCCCC1=CC2=CC(C3=CN(C45CC6CC(C5)CC(C4)C6)N=N3)=C7OP(O)(OC8=C(C7=C2C=C1)C9=CC=C(C=C9C=C8C%10=CN(C%11%12CC%13CC(C%12)CC(C%11)C%13)N=N%10)CCCCCCCC)=O'

In [7]:
big_mol = molecule(big_s)  # bad conformer id means that 0 conformers were generated by rdkit

ValueError: Bad Conformer Id

If you see errors like this, specify `large_mol=True` when you build molecule. Autoqchem will try something else

In [9]:
big_mol = molecule(big_s, large_mol=True)  # no more errors
draw(big_mol.mol)

interactive(children=(Dropdown(description='confId', options=(0, 1, 2, 3, 4), value=0), Output()), _dom_classe…

<function autoqchem.draw_utils._graph_conf(m, confId=0, energies=[])>

### Draw it!
It should look decent, OpenBabel performs MMFF94 optimization with a conformer search. If it doesn't look good, there might be issues with openbabel installation (this is rare, but can happen).

In [ ]:
draw(mols[0].mol)

### Test whether you can access SGE on Hoffman2 at UCLA
If you have an account on Hoffman2, change host to ```hoffman2.idre.ucla.edu```.

### Initialize the sge manager
change user parameter to your user

In [ ]:
sm = sge_manager(user='wang10', host='hoffman2.idre.ucla.edu')

### Verify that you can create an ssh tunnel into the remote server
Follow prompts for password and duo authentication. This tunnel will be used to manage jobs from within the notebook

In [ ]:
sm.connect()

### Check that you can see your jobs on the remote server
If the following command returns a table with your jobs currently running on the server, it's a success. If you have no jobs running, only a header row will be displayed.

In [ ]:
sm.qstat(summary=False)

### create job files locally

In [ ]:
for mol in mols:
    sm.create_jobs_for_molecule(mol, light_basis_set='6-31G**', solvent='DMSO')

### submit jobs to cluster

In [ ]:
sm.submit_jobs()

### check stat again, you can see all jobs submitted to cluster

In [ ]:
sm.qstat(summary=False)

### retrieve done jobs

In [ ]:
sm.retrieve_jobs()

### upload finished calculations

In [ ]:
sm.upload_done_molecules_to_db(tags=["test_WCG"])